In [1]:
from os import listdir
from os.path import isdir, join
import librosa
import random
import numpy as np
import matplotlib.pyplot as plt
import python_speech_features

In [2]:
! python3 -m pip install playsound

In [3]:
# set dataset path
dataset_path = './data_speech_commands_v002'

import os
# get speech_commands from google's server
if not os.path.exists(dataset_path):
    if not os.path.isfile('speech_commands_v0.02.tar.gz') :
        !wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
    !mkdir data_speech_commands_v002
    !tar zxvf "./speech_commands_v0.02.tar.gz" --directory data_speech_commands_v002
    
# Dataset path and view possible targets
dataset_path = dataset_path
for name in listdir(dataset_path):
    if isdir(join(dataset_path, name)):
        print(name)

nine
eight
bed
bird
no
house
go
zero
marvin
dog
off
_background_noise_
happy
stop
left
two
yes
on
learn
three
wow
tree
cat
right
down
six
five
backward
sheila
one
up
follow
visual
forward
seven
four


In [4]:
# Create an all targets list
all_targets = [name for name in listdir(dataset_path) if isdir(join(dataset_path, name))]
print(all_targets)

['nine', 'eight', 'bed', 'bird', 'no', 'house', 'go', 'zero', 'marvin', 'dog', 'off', '_background_noise_', 'happy', 'stop', 'left', 'two', 'yes', 'on', 'learn', 'three', 'wow', 'tree', 'cat', 'right', 'down', 'six', 'five', 'backward', 'sheila', 'one', 'up', 'follow', 'visual', 'forward', 'seven', 'four']


In [5]:
# Leave off background noise set
all_targets.remove('_background_noise_')
print(all_targets)

['nine', 'eight', 'bed', 'bird', 'no', 'house', 'go', 'zero', 'marvin', 'dog', 'off', 'happy', 'stop', 'left', 'two', 'yes', 'on', 'learn', 'three', 'wow', 'tree', 'cat', 'right', 'down', 'six', 'five', 'backward', 'sheila', 'one', 'up', 'follow', 'visual', 'forward', 'seven', 'four']


In [6]:
# See how many files are in each
num_samples = 0
for target in all_targets:
    print(len(listdir(join(dataset_path, target))))
    num_samples += len(listdir(join(dataset_path, target)))
print('Total samples:', num_samples)

3934
3787
2014
2064
3941
2113
3880
4052
2100
2128
3745
2054
3872
3801
3880
4044
3845
1575
3727
2123
1759
2031
3778
3917
3860
4052
1664
2022
3890
3723
1579
1592
1557
3998
3728
Total samples: 105829


In [7]:
# Settings
target_list = all_targets
feature_sets_file = 'all_targets_mfcc_sets.npz'
perc_keep_samples = 1.0 # 1.0 is keep all samples
val_ratio = 0.1
test_ratio = 0.1
sample_rate = 8000
num_mfcc = 16
len_mfcc = 16

In [8]:
# Create list of filenames along with ground truth vector (y)
filenames = []
y = []
for index, target in enumerate(target_list):
    print(join(dataset_path, target))
    filenames.append(listdir(join(dataset_path, target)))
    y.append(np.ones(len(filenames[index])) * index)

./data_speech_commands_v002/nine
./data_speech_commands_v002/eight
./data_speech_commands_v002/bed
./data_speech_commands_v002/bird
./data_speech_commands_v002/no
./data_speech_commands_v002/house
./data_speech_commands_v002/go
./data_speech_commands_v002/zero
./data_speech_commands_v002/marvin
./data_speech_commands_v002/dog
./data_speech_commands_v002/off
./data_speech_commands_v002/happy
./data_speech_commands_v002/stop
./data_speech_commands_v002/left
./data_speech_commands_v002/two
./data_speech_commands_v002/yes
./data_speech_commands_v002/on
./data_speech_commands_v002/learn
./data_speech_commands_v002/three
./data_speech_commands_v002/wow
./data_speech_commands_v002/tree
./data_speech_commands_v002/cat
./data_speech_commands_v002/right
./data_speech_commands_v002/down
./data_speech_commands_v002/six
./data_speech_commands_v002/five
./data_speech_commands_v002/backward
./data_speech_commands_v002/sheila
./data_speech_commands_v002/one
./data_speech_commands_v002/up
./data_speech

In [9]:
# Check ground truth Y vector
print(y)
for item in y:
    print(len(item))

[array([0., 0., 0., ..., 0., 0., 0.]), array([1., 1., 1., ..., 1., 1., 1.]), array([2., 2., 2., ..., 2., 2., 2.]), array([3., 3., 3., ..., 3., 3., 3.]), array([4., 4., 4., ..., 4., 4., 4.]), array([5., 5., 5., ..., 5., 5., 5.]), array([6., 6., 6., ..., 6., 6., 6.]), array([7., 7., 7., ..., 7., 7., 7.]), array([8., 8., 8., ..., 8., 8., 8.]), array([9., 9., 9., ..., 9., 9., 9.]), array([10., 10., 10., ..., 10., 10., 10.]), array([11., 11., 11., ..., 11., 11., 11.]), array([12., 12., 12., ..., 12., 12., 12.]), array([13., 13., 13., ..., 13., 13., 13.]), array([14., 14., 14., ..., 14., 14., 14.]), array([15., 15., 15., ..., 15., 15., 15.]), array([16., 16., 16., ..., 16., 16., 16.]), array([17., 17., 17., ..., 17., 17., 17.]), array([18., 18., 18., ..., 18., 18., 18.]), array([19., 19., 19., ..., 19., 19., 19.]), array([20., 20., 20., ..., 20., 20., 20.]), array([21., 21., 21., ..., 21., 21., 21.]), array([22., 22., 22., ..., 22., 22., 22.]), array([23., 23., 23., ..., 23., 23., 23.]), arr

In [10]:
# Flatten filename and y vectors
filenames = [item for sublist in filenames for item in sublist]
y = [item for sublist in y for item in sublist]

In [11]:
# Associate filenames with true output and shuffle
filenames_y = list(zip(filenames, y))
random.shuffle(filenames_y)
filenames, y = zip(*filenames_y)

In [12]:
# Only keep the specified number of samples (shorter extraction/training)
print(len(filenames))
filenames = filenames[:int(len(filenames) * perc_keep_samples)]
print(len(filenames))

105829
105829


In [13]:
# Calculate validation and test set sizes
val_set_size = int(len(filenames) * val_ratio)
test_set_size = int(len(filenames) * test_ratio)

In [14]:
# Break dataset apart into train, validation, and test sets
filenames_val = filenames[:val_set_size]
filenames_test = filenames[val_set_size:(val_set_size + test_set_size)]
filenames_train = filenames[(val_set_size + test_set_size):]

In [15]:
# Break y apart into train, validation, and test sets
y_orig_val = y[:val_set_size]
y_orig_test = y[val_set_size:(val_set_size + test_set_size)]
y_orig_train = y[(val_set_size + test_set_size):]

In [16]:
# Function: Create MFCC from given path
def calc_mfcc(path):
    
    # Load wavefile
    signal, fs = librosa.load(path, sr=sample_rate)
    
    # Create MFCCs from sound clip
    mfccs = python_speech_features.base.mfcc(signal, 
                                            samplerate=fs,
                                            winlen=0.256,
                                            winstep=0.050,
                                            numcep=num_mfcc,
                                            nfilt=26,
                                            nfft=2048,
                                            preemph=0.0,
                                            ceplifter=0,
                                            appendEnergy=False,
                                            winfunc=np.hanning)
    return mfccs.transpose()

In [17]:
# TEST: Construct test set by computing MFCC of each WAV file
prob_cnt = 0
x_test = []
y_test = []
for index, filename in enumerate(filenames_train):
    
    # Stop after 500
    if index >= 500:
        break
    
    # Create path from given filename and target item
    path = join(dataset_path, target_list[int(y_orig_train[index])], 
                filename)
    
    # Create MFCCs
    mfccs = calc_mfcc(path)
    
    if mfccs.shape[1] == len_mfcc:
        x_test.append(mfccs)
        y_test.append(y_orig_train[index])
    else:
        print('Dropped:', index, mfccs.shape)
        prob_cnt += 1
        

Dropped: 18 (16, 12)
Dropped: 41 (16, 7)
Dropped: 43 (16, 13)
Dropped: 51 (16, 13)
Dropped: 100 (16, 10)
Dropped: 103 (16, 13)
Dropped: 109 (16, 15)
Dropped: 112 (16, 13)
Dropped: 116 (16, 15)
Dropped: 129 (16, 11)
Dropped: 130 (16, 12)
Dropped: 133 (16, 12)
Dropped: 169 (16, 15)
Dropped: 170 (16, 13)
Dropped: 191 (16, 11)
Dropped: 198 (16, 9)
Dropped: 229 (16, 13)
Dropped: 234 (16, 15)
Dropped: 240 (16, 14)
Dropped: 256 (16, 7)
Dropped: 257 (16, 15)
Dropped: 271 (16, 13)
Dropped: 276 (16, 9)
Dropped: 303 (16, 15)
Dropped: 305 (16, 15)
Dropped: 357 (16, 11)
Dropped: 363 (16, 12)
Dropped: 403 (16, 13)
Dropped: 426 (16, 14)
Dropped: 428 (16, 11)
Dropped: 438 (16, 14)
Dropped: 454 (16, 6)
Dropped: 466 (16, 9)
Dropped: 474 (16, 12)
Dropped: 490 (16, 15)


In [18]:
print('% of problematic samples:', prob_cnt / 500)

% of problematic samples: 0.07


In [19]:
# TEST: Test shorter MFCC
# !pip install playsound
from playsound import playsound

idx = 13

# Create path from given filename and target item
path = join(dataset_path, target_list[int(y_orig_train[idx])], 
            filenames_train[idx])

# Create MFCCs
mfccs = calc_mfcc(path)
print("MFCCs:", mfccs)

# Plot MFCC
fig = plt.figure()
plt.imshow(mfccs, cmap='inferno', origin='lower')

# TEST: Play problem sounds
#print(target_list[int(y_orig_train[idx])])
#playsound(path)

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


MFCCs: [[-4.68040519e+01 -4.20935592e+01 -4.00342461e+01 -4.04233688e+01
  -4.28051255e+01 -4.20863068e+01 -3.83852202e+01 -3.58303140e+01
  -3.36559319e+01 -3.40492767e+01 -3.89869293e+01 -5.11707034e+01
  -6.37560692e+01 -7.11872000e+01 -7.76703854e+01 -8.20364479e+01]
 [ 2.27562743e+01  2.50567506e+01  2.60887569e+01  2.65285445e+01
   2.64659059e+01  2.44122095e+01  2.04447268e+01  1.66254761e+01
   1.15221889e+01  7.29767772e+00  4.20654180e+00  1.78365876e+00
   3.16840672e+00  4.12514170e+00  4.48721179e+00  4.59693295e+00]
 [ 5.18965491e+00  5.14602423e+00  4.94444482e+00  5.44538596e+00
   6.68890312e+00  5.37464407e+00  3.67006703e+00  3.80771881e+00
   4.90066083e+00  4.74459602e+00  3.47172811e+00  1.25189223e+00
  -3.49141193e-01  2.91652468e-01  1.55751076e+00  2.86406272e+00]
 [-6.83883114e+00 -8.49413266e+00 -8.57080822e+00 -7.46742945e+00
  -5.85817800e+00 -4.18672142e+00 -2.82613784e+00 -1.45725311e+00
   3.81155782e-01  7.37675750e-01  2.59160606e-01 -8.28352439e-01


In [20]:
# Function: Create MFCCs, keeping only ones of desired length
def extract_features(in_files, in_y):
    prob_cnt = 0
    out_x = []
    out_y = []
        
    for index, filename in enumerate(in_files):
    
        # Create path from given filename and target item
        path = join(dataset_path, target_list[int(in_y[index])], 
                    filename)
        
        # Check to make sure we're reading a .wav file
        if not path.endswith('.wav'):
            continue

        # Create MFCCs
        mfccs = calc_mfcc(path)

        # Only keep MFCCs with given length
        if mfccs.shape[1] == len_mfcc:
            out_x.append(mfccs)
            out_y.append(in_y[index])
        else:
            print('Dropped:', index, mfccs.shape)
            prob_cnt += 1
            
    return out_x, out_y, prob_cnt

In [21]:
# Create train, validation, and test sets
x_train, y_train, prob = extract_features(filenames_train, 
                                          y_orig_train)
print('Removed percentage:', prob / len(y_orig_train))
x_val, y_val, prob = extract_features(filenames_val, y_orig_val)
print('Removed percentage:', prob / len(y_orig_val))
x_test, y_test, prob = extract_features(filenames_test, y_orig_test)
print('Removed percentage:', prob / len(y_orig_test))

Dropped: 18 (16, 12)
Dropped: 41 (16, 7)
Dropped: 43 (16, 13)
Dropped: 51 (16, 13)
Dropped: 100 (16, 10)
Dropped: 103 (16, 13)
Dropped: 109 (16, 15)
Dropped: 112 (16, 13)
Dropped: 116 (16, 15)
Dropped: 129 (16, 11)
Dropped: 130 (16, 12)
Dropped: 133 (16, 12)
Dropped: 169 (16, 15)
Dropped: 170 (16, 13)
Dropped: 191 (16, 11)
Dropped: 198 (16, 9)
Dropped: 229 (16, 13)
Dropped: 234 (16, 15)
Dropped: 240 (16, 14)
Dropped: 256 (16, 7)
Dropped: 257 (16, 15)
Dropped: 271 (16, 13)
Dropped: 276 (16, 9)
Dropped: 303 (16, 15)
Dropped: 305 (16, 15)
Dropped: 357 (16, 11)
Dropped: 363 (16, 12)
Dropped: 403 (16, 13)
Dropped: 426 (16, 14)
Dropped: 428 (16, 11)
Dropped: 438 (16, 14)
Dropped: 454 (16, 6)
Dropped: 466 (16, 9)
Dropped: 474 (16, 12)
Dropped: 490 (16, 15)
Dropped: 507 (16, 11)
Dropped: 512 (16, 14)
Dropped: 521 (16, 12)
Dropped: 527 (16, 15)
Dropped: 540 (16, 14)
Dropped: 550 (16, 13)
Dropped: 567 (16, 11)
Dropped: 575 (16, 13)
Dropped: 576 (16, 9)
Dropped: 577 (16, 14)
Dropped: 582 (16, 12)

Dropped: 4389 (16, 15)
Dropped: 4398 (16, 7)
Dropped: 4420 (16, 13)
Dropped: 4436 (16, 7)
Dropped: 4444 (16, 14)
Dropped: 4448 (16, 10)
Dropped: 4450 (16, 11)
Dropped: 4479 (16, 10)
Dropped: 4488 (16, 11)
Dropped: 4499 (16, 11)
Dropped: 4502 (16, 15)
Dropped: 4542 (16, 13)
Dropped: 4592 (16, 10)
Dropped: 4628 (16, 15)
Dropped: 4647 (16, 13)
Dropped: 4652 (16, 11)
Dropped: 4660 (16, 13)
Dropped: 4665 (16, 14)
Dropped: 4693 (16, 14)
Dropped: 4704 (16, 15)
Dropped: 4731 (16, 15)
Dropped: 4733 (16, 11)
Dropped: 4749 (16, 7)
Dropped: 4774 (16, 9)
Dropped: 4780 (16, 9)
Dropped: 4785 (16, 14)
Dropped: 4794 (16, 11)
Dropped: 4801 (16, 7)
Dropped: 4810 (16, 12)
Dropped: 4816 (16, 15)
Dropped: 4821 (16, 15)
Dropped: 4827 (16, 10)
Dropped: 4829 (16, 15)
Dropped: 4832 (16, 13)
Dropped: 4837 (16, 9)
Dropped: 4840 (16, 13)
Dropped: 4846 (16, 14)
Dropped: 4860 (16, 15)
Dropped: 4866 (16, 12)
Dropped: 4872 (16, 15)
Dropped: 4875 (16, 12)
Dropped: 4881 (16, 11)
Dropped: 4892 (16, 10)
Dropped: 4925 (16,

Dropped: 8800 (16, 14)
Dropped: 8803 (16, 12)
Dropped: 8814 (16, 14)
Dropped: 8878 (16, 12)
Dropped: 8886 (16, 14)
Dropped: 8889 (16, 9)
Dropped: 8914 (16, 13)
Dropped: 8915 (16, 14)
Dropped: 8916 (16, 8)
Dropped: 8921 (16, 14)
Dropped: 8940 (16, 15)
Dropped: 8946 (16, 8)
Dropped: 8955 (16, 13)
Dropped: 8957 (16, 13)
Dropped: 8966 (16, 13)
Dropped: 8974 (16, 13)
Dropped: 8987 (16, 13)
Dropped: 9024 (16, 13)
Dropped: 9029 (16, 14)
Dropped: 9051 (16, 13)
Dropped: 9074 (16, 12)
Dropped: 9098 (16, 7)
Dropped: 9099 (16, 14)
Dropped: 9139 (16, 15)
Dropped: 9154 (16, 11)
Dropped: 9178 (16, 14)
Dropped: 9182 (16, 11)
Dropped: 9198 (16, 12)
Dropped: 9215 (16, 15)
Dropped: 9231 (16, 14)
Dropped: 9246 (16, 8)
Dropped: 9258 (16, 9)
Dropped: 9269 (16, 9)
Dropped: 9283 (16, 11)
Dropped: 9301 (16, 14)
Dropped: 9306 (16, 10)
Dropped: 9324 (16, 7)
Dropped: 9335 (16, 10)
Dropped: 9355 (16, 14)
Dropped: 9357 (16, 11)
Dropped: 9377 (16, 11)
Dropped: 9399 (16, 12)
Dropped: 9411 (16, 11)
Dropped: 9441 (16, 

Dropped: 13230 (16, 15)
Dropped: 13231 (16, 13)
Dropped: 13248 (16, 11)
Dropped: 13249 (16, 12)
Dropped: 13258 (16, 12)
Dropped: 13264 (16, 13)
Dropped: 13283 (16, 10)
Dropped: 13295 (16, 13)
Dropped: 13296 (16, 12)
Dropped: 13301 (16, 11)
Dropped: 13305 (16, 13)
Dropped: 13316 (16, 12)
Dropped: 13319 (16, 13)
Dropped: 13330 (16, 14)
Dropped: 13337 (16, 15)
Dropped: 13339 (16, 12)
Dropped: 13341 (16, 11)
Dropped: 13345 (16, 15)
Dropped: 13350 (16, 10)
Dropped: 13372 (16, 7)
Dropped: 13373 (16, 11)
Dropped: 13382 (16, 13)
Dropped: 13393 (16, 15)
Dropped: 13400 (16, 13)
Dropped: 13404 (16, 13)
Dropped: 13431 (16, 12)
Dropped: 13437 (16, 15)
Dropped: 13451 (16, 8)
Dropped: 13462 (16, 11)
Dropped: 13484 (16, 12)
Dropped: 13492 (16, 8)
Dropped: 13513 (16, 9)
Dropped: 13518 (16, 14)
Dropped: 13528 (16, 14)
Dropped: 13542 (16, 12)
Dropped: 13543 (16, 14)
Dropped: 13556 (16, 9)
Dropped: 13569 (16, 10)
Dropped: 13604 (16, 11)
Dropped: 13606 (16, 13)
Dropped: 13610 (16, 15)
Dropped: 13622 (16, 1

Dropped: 16807 (16, 15)
Dropped: 16823 (16, 14)
Dropped: 16825 (16, 11)
Dropped: 16827 (16, 14)
Dropped: 16828 (16, 6)
Dropped: 16830 (16, 11)
Dropped: 16841 (16, 10)
Dropped: 16862 (16, 11)
Dropped: 16873 (16, 13)
Dropped: 16875 (16, 13)
Dropped: 16878 (16, 13)
Dropped: 16879 (16, 15)
Dropped: 16887 (16, 13)
Dropped: 16890 (16, 11)
Dropped: 16894 (16, 13)
Dropped: 16906 (16, 11)
Dropped: 16910 (16, 15)
Dropped: 16918 (16, 15)
Dropped: 16922 (16, 13)
Dropped: 16924 (16, 8)
Dropped: 16943 (16, 7)
Dropped: 16949 (16, 12)
Dropped: 16958 (16, 10)
Dropped: 16969 (16, 11)
Dropped: 16970 (16, 15)
Dropped: 16976 (16, 13)
Dropped: 16996 (16, 11)
Dropped: 17010 (16, 11)
Dropped: 17016 (16, 15)
Dropped: 17019 (16, 13)
Dropped: 17026 (16, 15)
Dropped: 17027 (16, 13)
Dropped: 17042 (16, 10)
Dropped: 17057 (16, 8)
Dropped: 17065 (16, 10)
Dropped: 17077 (16, 13)
Dropped: 17085 (16, 15)
Dropped: 17098 (16, 11)
Dropped: 17102 (16, 10)
Dropped: 17107 (16, 14)
Dropped: 17110 (16, 13)
Dropped: 17120 (16, 

...

Dropped: 1495 (16, 15)
Dropped: 1507 (16, 13)
Dropped: 1511 (16, 15)
Dropped: 1517 (16, 10)
Dropped: 1533 (16, 15)
Dropped: 1539 (16, 9)
Dropped: 1540 (16, 13)
Dropped: 1562 (16, 13)
Dropped: 1603 (16, 3)
Dropped: 1605 (16, 15)
Dropped: 1654 (16, 13)
Dropped: 1667 (16, 10)
Dropped: 1685 (16, 12)
Dropped: 1701 (16, 11)
Dropped: 1703 (16, 15)
Dropped: 1730 (16, 15)
Dropped: 1733 (16, 12)
Dropped: 1739 (16, 13)
Dropped: 1748 (16, 13)
Dropped: 1762 (16, 12)
Dropped: 1765 (16, 15)
Dropped: 1786 (16, 15)
Dropped: 1804 (16, 15)
Dropped: 1808 (16, 13)
Dropped: 1827 (16, 13)
Dropped: 1860 (16, 6)
Dropped: 1875 (16, 13)
Dropped: 1898 (16, 9)
Dropped: 1901 (16, 14)
Dropped: 1903 (16, 13)
Dropped: 1912 (16, 10)
Dropped: 1957 (16, 8)
Dropped: 1967 (16, 10)
Dropped: 1969 (16, 14)
Dropped: 1986 (16, 5)
Dropped: 1989 (16, 14)
Dropped: 1992 (16, 14)
Dropped: 2011 (16, 14)
Dropped: 2019 (16, 13)
Dropped: 2039 (16, 12)
Dropped: 2048 (16, 15)
Dropped: 2077 (16, 13)
Dropped: 2086 (16, 13)
Dropped: 2092 (16

Dropped: 5908 (16, 6)
Dropped: 5917 (16, 13)
Dropped: 5919 (16, 13)
Dropped: 5937 (16, 5)
Dropped: 5956 (16, 15)
Dropped: 5981 (16, 14)
Dropped: 5991 (16, 3)
Dropped: 6000 (16, 13)
Dropped: 6036 (16, 13)
Dropped: 6066 (16, 15)
Dropped: 6072 (16, 11)
Dropped: 6084 (16, 11)
Dropped: 6086 (16, 11)
Dropped: 6115 (16, 8)
Dropped: 6127 (16, 13)
Dropped: 6130 (16, 13)
Dropped: 6198 (16, 15)
Dropped: 6203 (16, 11)
Dropped: 6210 (16, 13)
Dropped: 6236 (16, 15)
Dropped: 6252 (16, 15)
Dropped: 6264 (16, 13)
Dropped: 6267 (16, 15)
Dropped: 6285 (16, 15)
Dropped: 6290 (16, 13)
Dropped: 6299 (16, 15)
Dropped: 6300 (16, 7)
Dropped: 6306 (16, 11)
Dropped: 6327 (16, 13)
Dropped: 6331 (16, 14)
Dropped: 6340 (16, 13)
Dropped: 6356 (16, 12)
Dropped: 6371 (16, 12)
Dropped: 6377 (16, 11)
Dropped: 6383 (16, 11)
Dropped: 6384 (16, 13)
Dropped: 6399 (16, 15)
Dropped: 6405 (16, 9)
Dropped: 6423 (16, 6)
Dropped: 6427 (16, 14)
Dropped: 6460 (16, 11)
Dropped: 6489 (16, 7)
Dropped: 6501 (16, 15)
Dropped: 6502 (16, 

Dropped: 9861 (16, 10)
Dropped: 9886 (16, 9)
Dropped: 9895 (16, 11)
Dropped: 9898 (16, 13)
Dropped: 9906 (16, 14)
Dropped: 9920 (16, 14)
Dropped: 9927 (16, 9)
Dropped: 9964 (16, 14)
Dropped: 9967 (16, 11)
Dropped: 9972 (16, 11)
Dropped: 9978 (16, 14)
Dropped: 9981 (16, 11)
Dropped: 9985 (16, 11)
Dropped: 10002 (16, 15)
Dropped: 10017 (16, 12)
Dropped: 10024 (16, 15)
Dropped: 10027 (16, 11)
Dropped: 10030 (16, 13)
Dropped: 10051 (16, 13)
Dropped: 10081 (16, 15)
Dropped: 10110 (16, 8)
Dropped: 10113 (16, 14)
Dropped: 10125 (16, 8)
Dropped: 10132 (16, 11)
Dropped: 10142 (16, 12)
Dropped: 10149 (16, 10)
Dropped: 10158 (16, 12)
Dropped: 10173 (16, 13)
Dropped: 10177 (16, 12)
Dropped: 10186 (16, 11)
Dropped: 10190 (16, 12)
Dropped: 10195 (16, 14)
Dropped: 10196 (16, 8)
Dropped: 10203 (16, 13)
Dropped: 10217 (16, 13)
Dropped: 10224 (16, 15)
Dropped: 10227 (16, 12)
Dropped: 10229 (16, 11)
Dropped: 10232 (16, 7)
Dropped: 10233 (16, 8)
Dropped: 10242 (16, 10)
Dropped: 10247 (16, 11)
Dropped: 102

Dropped: 3482 (16, 14)
Dropped: 3492 (16, 13)
Dropped: 3505 (16, 10)
Dropped: 3515 (16, 15)
Dropped: 3520 (16, 14)
Dropped: 3528 (16, 15)
Dropped: 3530 (16, 10)
Dropped: 3543 (16, 12)
Dropped: 3550 (16, 14)
Dropped: 3600 (16, 8)
Dropped: 3603 (16, 13)
Dropped: 3622 (16, 13)
Dropped: 3627 (16, 7)
Dropped: 3630 (16, 14)
Dropped: 3631 (16, 10)
Dropped: 3634 (16, 12)
Dropped: 3649 (16, 13)
Dropped: 3673 (16, 15)
Dropped: 3692 (16, 14)
Dropped: 3696 (16, 12)
Dropped: 3699 (16, 13)
Dropped: 3700 (16, 12)
Dropped: 3712 (16, 12)
Dropped: 3715 (16, 13)
Dropped: 3723 (16, 13)
Dropped: 3724 (16, 14)
Dropped: 3726 (16, 14)
Dropped: 3731 (16, 13)
Dropped: 3753 (16, 10)
Dropped: 3770 (16, 14)
Dropped: 3796 (16, 8)
Dropped: 3797 (16, 15)
Dropped: 3799 (16, 15)
Dropped: 3804 (16, 12)
Dropped: 3826 (16, 12)
Dropped: 3833 (16, 12)
Dropped: 3842 (16, 14)
Dropped: 3865 (16, 11)
Dropped: 3868 (16, 11)
Dropped: 3874 (16, 13)
Dropped: 3881 (16, 12)
Dropped: 3888 (16, 15)
Dropped: 3905 (16, 14)
Dropped: 3910 

Dropped: 7751 (16, 15)
Dropped: 7766 (16, 11)
Dropped: 7776 (16, 13)
Dropped: 7800 (16, 15)
Dropped: 7807 (16, 14)
Dropped: 7811 (16, 7)
Dropped: 7813 (16, 8)
Dropped: 7820 (16, 14)
Dropped: 7835 (16, 6)
Dropped: 7849 (16, 14)
Dropped: 7895 (16, 11)
Dropped: 7898 (16, 7)
Dropped: 7962 (16, 10)
Dropped: 7964 (16, 13)
Dropped: 7971 (16, 15)
Dropped: 7985 (16, 14)
Dropped: 8014 (16, 10)
Dropped: 8022 (16, 5)
Dropped: 8023 (16, 13)
Dropped: 8034 (16, 13)
Dropped: 8035 (16, 8)
Dropped: 8041 (16, 10)
Dropped: 8050 (16, 14)
Dropped: 8054 (16, 7)
Dropped: 8056 (16, 12)
Dropped: 8059 (16, 13)
Dropped: 8071 (16, 10)
Dropped: 8074 (16, 10)
Dropped: 8084 (16, 8)
Dropped: 8094 (16, 15)
Dropped: 8095 (16, 14)
Dropped: 8102 (16, 13)
Dropped: 8103 (16, 15)
Dropped: 8121 (16, 11)
Dropped: 8123 (16, 9)
Dropped: 8154 (16, 13)
Dropped: 8167 (16, 13)
Dropped: 8171 (16, 15)
Dropped: 8179 (16, 7)
Dropped: 8182 (16, 13)
Dropped: 8193 (16, 13)
Dropped: 8194 (16, 14)
Dropped: 8201 (16, 14)
Dropped: 8224 (16, 14

In [22]:
# Save features and truth vector (y) sets to disk
np.savez(feature_sets_file, 
         x_train=x_train, 
         y_train=y_train, 
         x_val=x_val, 
         y_val=y_val, 
         x_test=x_test, 
         y_test=y_test)

In [23]:
# TEST: Load features
feature_sets = np.load(feature_sets_file)
feature_sets.files

['x_train', 'y_train', 'x_val', 'y_val', 'x_test', 'y_test']

In [24]:
len(feature_sets['x_train'])

77497

In [25]:
print(feature_sets['y_val'])

[12. 34. 30. ... 19. 18. 32.]
